In [1]:
import numpy as np                     # numpy
import pandas as pd                    # pandas

from numpy.linalg import norm

In [2]:
#import implementation of task 1 for reuse
%run ../Task1/Implementation.ipynb

#your code here
def calculate_cosine_similarity(inputDataFrame: pd.DataFrame) -> pd.DataFrame:
    individualTermFrequencies = inputDataFrame["individual tf"] #get tfs
    
    # we have list of terms, with lists of tfs in documents.
    # we want list of documents, with lists of tf for terms
    documentVector = list(map(list, zip(*individualTermFrequencies))) #'transpose' the individual tf matrix, so that we get documents
    
    # calculate cosine similarity for each pair
    cosineSimilarityMatrix = []
    for A in documentVector:
        cosineSimilarityList = []
        for B in documentVector:
            cosineSimlarity = np.dot(A,B)/(norm(A)*norm(B))
            cosineSimilarityList.append(cosineSimlarity)
        cosineSimilarityMatrix.append(cosineSimilarityList)
    
    df = pd.DataFrame(cosineSimilarityMatrix)
    df.index += 1 #increment the indexes
    for i in reversed(range(len(documentVector))): # reversed because else, it names all columns the same
        df.rename(columns={i: i+1}, inplace=True) 
    
    return df

In [3]:
calculate_cosine_similarity(calculate_tfidf('../collection1.txt'))

,1,2,3,4,5
1,1.000000,0.572892,0.780720,0.828079,0.155700
2,0.572892,1.000000,0.733799,0.889499,0.501745
3,0.780720,0.733799,1.000000,0.808122,0.341882
4,0.828079,0.889499,0.808122,1.000000,0.402911
5,0.155700,0.501745,0.341882,0.402911,1.000000
